<div style="text-align: right"> Mateusz Kobyłka (Berlin), 268869 </div>
<h1><center>PADPy 2018/2019</center></h1> 
<h3><center>Praca domowa nr 2</center></h3>

Praca domowa nr 2 składa się z 7 zadań polegających na pobraniu pewnych informacji ze zbioru zanonimizowanych danych z serwisu https://travel.stackexchange.com/ . Zbiór ten tworzą następujące ramki danych: 
* Badges, 
* Comments, 
* PostLinks, 
* Posts, 
* Tags, 
* Users, 
* Votes. 

Znaczenie poszczególnych kolumn można znaleźć w pliku http://www.gagolewski.com/resources/data/travel_stackexchange_com/readme.txt. 
Każde z zadań polega na pobraniu informacji zapisanych w jego treści (w języku SQL) na dwa równoważne sposoby, kolejno:
1. wywołanie *pandas.read_sql_query("""zapytanie SQL""")*;
2. wywołanie ciągu „zwykłych” metod i funkcji z pakietu **pandas**.

Przed przystąpieniem do wykonywania zadań wczytane zostaną wymagane pakiety oraz dane.

In [1]:
import pandas as pd
import sqlite3

Dane zostaną wczytane bezpośrednio ze strony http://www.gagolewski.com/resources/data/travel_stackexchange_com/ za pomocą funkcji *read_csv*.

In [2]:
link = "http://www.gagolewski.com/resources/data/travel_stackexchange_com/"

Badges = pd.read_csv(link + "Badges.csv.gz",comment="#", compression="gzip")
Comments = pd.read_csv(link + "Comments.csv.gz",comment="#", compression="gzip")
PostLinks = pd.read_csv(link + "PostLinks.csv.gz",comment="#", compression="gzip")
Posts = pd.read_csv(link + "Posts.csv.gz",comment="#", compression="gzip")
Tags = pd.read_csv(link + "Tags.csv.gz",comment="#", compression="gzip")
Users = pd.read_csv(link + "Users.csv.gz",comment="#", compression="gzip")
Votes = pd.read_csv(link + "Votes.csv.gz",comment="#", compression="gzip")

Każda z ramek danych zostanie również wyeksportowana do bazy danych SQLite *travel.db*. Umożliwia to metoda *to_sql*.

In [3]:
conn = sqlite3.connect("travel.db")

In [4]:
Badges.to_sql("Badges",conn)
Comments.to_sql("Comments",conn)
PostLinks.to_sql("PostLinks",conn)
Posts.to_sql("Posts",conn)
Tags.to_sql("Tags",conn)
Users.to_sql("Users",conn)
Votes.to_sql("Votes",conn)

W każdym z zadań, oprócz  zaprezentowania wynikowych tabel wywołana zostanie również funckja *compare_results(first,second)*, która sprawdza, czy otrzymane wyniki są rzeczywiście klasy **DataFrame** oraz czy zwracane zbiory danych są tożsame. Poniżej znajduje się jej kod źródłowy.

In [5]:
def compare_results(first,second):
    if not isinstance(first,pd.core.frame.DataFrame) or not isinstance(first,pd.core.frame.DataFrame):
        print('Przekazane argumenty nie są klasy DataFrame')
        return
    
    if first.equals(second):
        print('Ramki danych są tożsame!')
    else:
        print('Ramki danych nie są tożsame!')

<h1></h1>
<h2><center>Zadanie 1</center></h2>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Zapytanie SQL z zadania pierwszego zwraca 10 użytkowników serwisu Stack Exchange, których pytania otrzymały najwięcej głosów typu _Favorite_ (tzn. inni użytkownicy dodali zadane przez nich pytania do tzw. *ulubionych pytań*). Oprócz nazwy użytkownika (*DisplayName*), zapytanie zwraca jego wiek (*Age*) i miejsce zamieszkania (*Location*) oraz sumę głosów typu *Favorite* dla wszystkich zadanych pytań (*FavoriteTotal*), pytanie, które zostało najczęściej dodane do ulubionych przez innych użytkowników (*MostFavoriteQuestion*) oraz ich liczbę (*MostFavoriteQuestionLikes*). Wynikowa tabela uporządkowana jest malejąco względem kolumny *FavoriteTotal*.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Poniżej znajduje się opisane zapytanie SQL, a w kolejnych punktach:
* wywołanie *pandas.read_sql_query* oraz wynikowa tabela,
* wywołanie ciągu ,,zwykłych" funkcji i metod z pakietu **pandas**,
* porównanie wyników przy pomocy funkcji *compare_results*.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Każde z kolejnych zadań rozwiązane będzie według tego schematu.

```SQL
SELECT
    Users.DisplayName,
    Users.Age,
    Users.Location,
    SUM(Posts.FavoriteCount) AS FavoriteTotal,
    Posts.Title AS MostFavoriteQuestion,
    MAX(Posts.FavoriteCount) AS MostFavoriteQuestionLikes
FROM Posts
JOIN Users ON Users.Id=Posts.OwnerUserId
WHERE Posts.PostTypeId=1
GROUP BY OwnerUserId
ORDER BY FavoriteTotal DESC
LIMIT 10
```

**1.1.** pd.read_sql_query



In [6]:
result_1_1 =pd.read_sql_query("""
SELECT
    Users.DisplayName,
    Users.Age,
    Users.Location,
    SUM(Posts.FavoriteCount) AS FavoriteTotal,
    Posts.Title AS MostFavoriteQuestion,
    MAX(Posts.FavoriteCount) AS MostFavoriteQuestionLikes
FROM Posts
JOIN Users ON Users.Id=Posts.OwnerUserId
WHERE Posts.PostTypeId=1
GROUP BY OwnerUserId
ORDER BY FavoriteTotal DESC
LIMIT 10
""", conn)
result_1_1

,DisplayName,Age,Location,FavoriteTotal,MostFavoriteQuestion,MostFavoriteQuestionLikes
0,Mark Mayo,37.0,"Sydney, New South Wales, Australia",467.0,Tactics to avoid getting harassed by corrupt p...,42.0
1,hippietrail,NaN,"Oaxaca, Mexico",444.0,"OK we're all adults here, so really, how on ea...",79.0
2,RoflcoptrException,NaN,None,294.0,How to avoid drinking vodka?,29.0
3,JonathanReez,26.0,"Prague, Czech Republic",221.0,What is the highest viewing spot in London tha...,17.0
4,nsn,NaN,None,214.0,How do airlines determine ticket prices?,40.0
5,Gagravarr,NaN,"Oxford, United Kingdom",151.0,Are there other places with gardens like those...,10.0
6,Andrew Grimm,38.0,"Sydney, Australia",120.0,"OK we're all nerds here, so really, how on ear...",8.0
7,VMAtm,33.0,"Tampa, FL, United States",109.0,Is there a good website to plan a trip via tra...,34.0
8,jrdioko,NaN,None,100.0,What is the most comfortable way to sleep on a...,21.0
9,Gayot Fow,NaN,"London, United Kingdom",98.0,Should I submit bank statements when applying ...,18.0


**1.2.** pandas

In [7]:
result_1_2 = (Posts.loc[Posts.loc[:,'PostTypeId']==1,:].sort_values(by='FavoriteCount',ascending=False)
              .groupby(['OwnerUserId'])
              .agg({'FavoriteCount':  ['sum','max'], 'Title': 'first'})
              .reset_index()
              .rename(columns={'sum':'FavoriteTotal', 'max': 'MostFavoriteQuestionLikes', 'first': 'MostFavoriteQuestion'}))
result_1_2.columns = result_1_2.columns.droplevel()
result_1_2 = result_1_2.sort_values(by='FavoriteTotal', ascending = False).head(10)\
                       .merge(Users,left_on='', right_on = 'Id')\
                       .loc[:,['DisplayName','Age','Location','FavoriteTotal',
                               'MostFavoriteQuestion','MostFavoriteQuestionLikes']]
result_1_2

,DisplayName,Age,Location,FavoriteTotal,MostFavoriteQuestion,MostFavoriteQuestionLikes
0,Mark Mayo,37.0,"Sydney, New South Wales, Australia",467.0,Tactics to avoid getting harassed by corrupt p...,42.0
1,hippietrail,NaN,"Oaxaca, Mexico",444.0,"OK we're all adults here, so really, how on ea...",79.0
2,RoflcoptrException,NaN,NaN,294.0,How to avoid drinking vodka?,29.0
3,JonathanReez,26.0,"Prague, Czech Republic",221.0,What is the highest viewing spot in London tha...,17.0
4,nsn,NaN,NaN,214.0,How do airlines determine ticket prices?,40.0
5,Gagravarr,NaN,"Oxford, United Kingdom",151.0,Are there other places with gardens like those...,10.0
6,Andrew Grimm,38.0,"Sydney, Australia",120.0,"OK we're all nerds here, so really, how on ear...",8.0
7,VMAtm,33.0,"Tampa, FL, United States",109.0,Is there a good website to plan a trip via tra...,34.0
8,jrdioko,NaN,NaN,100.0,What is the most comfortable way to sleep on a...,21.0
9,Gayot Fow,NaN,"London, United Kingdom",98.0,Should I submit bank statements when applying ...,18.0


**1.3** Porównanie wyników

In [8]:
compare_results(result_1_1,result_1_2)

Ramki danych są tożsame!


<h1></h1>
<h2><center>Zadanie 2</center></h2>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Drugie zapytanie zwraca pytania z największą liczbą pozytywnie ocenionych odpowiedzi. Każda z odpowiedzi udzielona przez użytkowników może zostać oceniona przez innych jako przydatna (tzw. _łapka w górę_) lub nie (_łapka w dół_). Na podstawie *łapek* wyliczany jest _Score_ (jako różnica _łapek w góre_ i _łapek w dół_) - jeśli jego wartość jest dodatnia, to taką odpowiedź można uznać za pozytywnie ocenioną. 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Zapytanie zwraca 10 pytań z największą liczbą pozytywnie ocenionych odpowiedzi. W wynikowych tabelach, oprócz Id pytania (jego numeru w bazie), podane jest również samo pytanie (_Title_) oraz liczba pozytywnie ocenionych odpowiedzi (_PositiveAnswerCount_). Tabela posortowana jest malejąco względem ostatniej kolumny.

```SQL
SELECT
    Posts.ID,
    Posts.Title,
    Posts2.PositiveAnswerCount
FROM Posts
JOIN (
        SELECT
            Posts.ParentID,
            COUNT(*) AS PositiveAnswerCount
        FROM Posts
        WHERE Posts.PostTypeID=2 AND Posts.Score>0
        GROUP BY Posts.ParentID
    ) AS Posts2
    ON Posts.ID=Posts2.ParentID
ORDER BY Posts2.PositiveAnswerCount DESC
LIMIT 10
```

**2.1.** pd.read_sql_query

In [9]:
result_2_1 = pd.read_sql_query("""
SELECT
    Posts.ID,
    Posts.Title,
    Posts2.PositiveAnswerCount
FROM Posts
JOIN (
        SELECT
            Posts.ParentID,
            COUNT(*) AS PositiveAnswerCount
        FROM Posts
        WHERE Posts.PostTypeID=2 AND Posts.Score>0
        GROUP BY Posts.ParentID
    ) AS Posts2
    ON Posts.ID=Posts2.ParentID
ORDER BY Posts2.PositiveAnswerCount DESC
LIMIT 10
""", conn)
result_2_1

,Id,Title,PositiveAnswerCount
0,250,Which European cities have bike rental station...,24
1,10,When traveling to a country with a different c...,20
2,13562,How do you choose a restaurant when travelling?,20
3,48775,How can I deal with people asking to switch se...,20
4,3220,Why would you wrap your luggage in plastic?,19
5,43660,Traveling in Europe Solo - 18 years old. Feasi...,19
6,30656,Long-life SIM cards in Europe,18
7,7663,Am I expected to tip wait staff in Europe?,17
8,59128,"Is there a way to prevent ""looking like a tour...",17
9,60446,Is it rude to ask if the food contains pork or...,17


**2.2.** pandas

In [10]:
Posts2 = Posts.loc[Posts.loc[:,'Score'] > 0,:].groupby(['ParentId'])['Id'].agg('count').reset_index()\
              .rename(columns={'Id': 'PositiveAnswerCount'})
result_2_2 = pd.merge(Posts,Posts2, left_on = 'Id', right_on = 'ParentId').loc[:,['Id','Title','PositiveAnswerCount']]\
               .sort_values(by=['PositiveAnswerCount','Id'], ascending = [False,True]).reset_index(drop=True).head(10)
result_2_2

,Id,Title,PositiveAnswerCount
0,250,Which European cities have bike rental station...,24
1,10,When traveling to a country with a different c...,20
2,13562,How do you choose a restaurant when travelling?,20
3,48775,How can I deal with people asking to switch se...,20
4,3220,Why would you wrap your luggage in plastic?,19
5,43660,Traveling in Europe Solo - 18 years old. Feasi...,19
6,30656,Long-life SIM cards in Europe,18
7,7663,Am I expected to tip wait staff in Europe?,17
8,59128,"Is there a way to prevent ""looking like a tour...",17
9,60446,Is it rude to ask if the food contains pork or...,17


**2.3.** Porównanie wyników

In [11]:
compare_results(result_2_1,result_2_2)

Ramki danych są tożsame!


<h1></h1>
<h2><center>Zadanie 3</center></h2>
    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Zapytanie z zadania trzeciego zwraca pytania z największą liczbą _łapek w górę_ na przestrzeni lat. Oprócz samego pytania (_Title_) zwracany jest rok, w którym zostało ono zadane (_Year_) oraz liczba pozytywnych ocen (_łapek w górę_), którą dane pytanie otrzymało (_Count_). Dla każdego roku zwracane jest jedno, najlepsze pytanie, dlatego też tabela wynikowa może być potraktowana jako zestaw _pytań roku_ pod względem liczby pozytywnych ocen. 

```SQL
SELECT
    Posts.Title,
    UpVotesPerYear.Year,
    MAX(UpVotesPerYear.Count) AS Count
FROM (
       SELECT
            PostId,
            COUNT(*) AS Count,
            STRFTIME('%Y', Votes.CreationDate) AS Year
        FROM Votes
        WHERE VoteTypeId=2
        GROUP BY PostId, Year
        ) AS UpVotesPerYear
JOIN Posts ON Posts.Id=UpVotesPerYear.PostId
WHERE Posts.PostTypeId=1
GROUP BY Year
```

**3.1.** pd.read_sql_query

In [12]:
result_3_1 = pd.read_sql_query("""
SELECT
    Posts.Title,
    UpVotesPerYear.Year,
    MAX(UpVotesPerYear.Count) AS Count
FROM (
       SELECT
            PostId,
            COUNT(*) AS Count,
            STRFTIME('%Y', Votes.CreationDate) AS Year
        FROM Votes
        WHERE VoteTypeId=2
        GROUP BY PostId, Year
        ) AS UpVotesPerYear
JOIN Posts ON Posts.Id=UpVotesPerYear.PostId
WHERE Posts.PostTypeId=1
GROUP BY Year
""", conn)
result_3_1

,Title,Year,Count
0,"OK we're all adults here, so really, how on ea...",2011,70
1,How to successfully haggle / bargain in markets,2012,37
2,Why are airline passengers asked to lift up wi...,2013,103
3,How do you know if Americans genuinely/literal...,2014,180
4,Immigration officer that stopped me at the air...,2015,119
5,I don't know my nationality. How can I visit D...,2016,135
6,Why prohibit engine braking?,2017,178


**3.2.** pandas

In [13]:
Votes.loc[:,'Year'] = pd.to_datetime(Votes.loc[:,'CreationDate']).apply(lambda x: x.year).astype(str)
UpVotesPerYear = Votes.query('VoteTypeId==2').groupby(['PostId','Year'])['Id'].agg('count').reset_index()\
                      .rename(columns={'Id':'Count'})
result_3_2 = pd.merge(UpVotesPerYear,Posts,left_on = 'PostId', right_on = 'Id').sort_values('Count',ascending=False)\
               .query('PostTypeId==1').groupby('Year').agg({'Count': 'max', 'Title': 'first'}).reset_index()\
               .loc[:,['Title','Year','Count']]
result_3_2

,Title,Year,Count
0,"OK we're all adults here, so really, how on ea...",2011,70
1,How to successfully haggle / bargain in markets,2012,37
2,Why are airline passengers asked to lift up wi...,2013,103
3,How do you know if Americans genuinely/literal...,2014,180
4,Immigration officer that stopped me at the air...,2015,119
5,I don't know my nationality. How can I visit D...,2016,135
6,Why prohibit engine braking?,2017,178


**3.3** Porównanie wyników

In [14]:
compare_results(result_3_1,result_3_2)

Ramki danych są tożsame!


<h1></h1>
<h2><center>Zadanie 4</center></h2>
 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Zadanie czwarte zwraca ciekawą statystykę pokazującą różnice w głosach między najlepszą odpowiedzią pod względem liczby głosów i najlepszą odpowiedzią wybraną przez pytającego. Każdy użytkownik po zadaniu pytania może spośród udzielonych odpowiedzi wybrać tę, która była dla niego najbardziej pomocna i wyróżnić ją. Taka opdowiedź będzie znajdowała się bezpośrednio pod zadanym pytaniem.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Oczywiście, inni użytkownicy mogą nadal głosować przy pomocy _łapek_ na zarówno najlepszą odpowiedź według pytającego, jak i na inne odpowiedzi. Różnica w głosach między najlepszą odpowiedzią według użytkowników oraz najlepszą odpowiedzią według pytającego to jedna z kolumn w wynikowej ramce danych (_Difference_). Oprócz niej, zwracane jest również Id pytania (jego numer w bazie), treść (_Title_), _Score_ najlepszej odpowiedzi według użytkowników (_MaxScore_) oraz _Score_ najlepszej odpowiedzi według pytającego. W tabeli zostały pokazane tylko te pytania, dla których wspomniana wcześniej różnica (_Difference_) jest większa niż 50 głosów.

```SQL
SELECT
    Questions.Id,
    Questions.Title,
    BestAnswers.MaxScore,
    Posts.Score AS AcceptedScore,
    BestAnswers.MaxScore-Posts.Score AS Difference
FROM (
        SELECT Id, ParentId, MAX(Score) AS MaxScore
        FROM Posts
        WHERE PostTypeId==2
        GROUP BY ParentId
    ) AS BestAnswers
JOIN (
        SELECT * FROM Posts
        WHERE PostTypeId==1
    ) AS Questions
    ON Questions.Id=BestAnswers.ParentId
JOIN Posts ON Questions.AcceptedAnswerId=Posts.Id
WHERE Difference>50
ORDER BY Difference DESC
```

**4.1.** pd.read_sql_query

In [15]:
result_4_1 = pd.read_sql_query("""
SELECT
    Questions.Id,
    Questions.Title,
    BestAnswers.MaxScore,
    Posts.Score AS AcceptedScore,
    BestAnswers.MaxScore-Posts.Score AS Difference
FROM (
        SELECT Id, ParentId, MAX(Score) AS MaxScore
        FROM Posts
        WHERE PostTypeId==2
        GROUP BY ParentId
    ) AS BestAnswers
JOIN (
        SELECT * FROM Posts
        WHERE PostTypeId==1
    ) AS Questions
    ON Questions.Id=BestAnswers.ParentId
JOIN Posts ON Questions.AcceptedAnswerId=Posts.Id
WHERE Difference>50
ORDER BY Difference DESC
""", conn)
result_4_1

,Id,Title,MaxScore,AcceptedScore,Difference
0,99713,"In Germany, what are you supposed to do if you...",126,33,93
1,81376,What is way to eat rice with hands in front of...,120,30,90
2,76737,Why do many countries in the world still requi...,118,31,87
3,38177,How do you know if Americans genuinely/literal...,245,166,79
4,90636,Caught speeding 111 Mph (179 km/h) in Californ...,101,25,76
5,81492,Why is international first class much more exp...,90,21,69
6,94320,Strange looking region of France on Google Maps,71,7,64
7,80050,What's an easy way of making my luggage unique...,72,16,56


**4.2.** pandas

In [16]:
BestAnswers = Posts.query('PostTypeId==2').groupby(['ParentId']).agg({'Score': 'max', 'Id': 'first'}).reset_index()\
                   .rename(columns={'Score': 'MaxScore'})
Questions = Posts.loc[Posts.loc[:,'PostTypeId']==1,:]
result_4_2 = BestAnswers.merge(Questions, left_on = 'ParentId', right_on='Id')\
                        .merge(Posts,left_on = 'AcceptedAnswerId', right_on = 'Id')
result_4_2.loc[:,'Difference'] = result_4_2.loc[:,'MaxScore']-result_4_2.loc[:,'Score_y']
result_4_2 = result_4_2.loc[result_4_2.loc[:,'Difference']>50,['Id_y','Title_x','MaxScore','Score_y','Difference']]\
                       .rename(columns={'Id_y': 'Id', 'Title_x': 'Title','Score_y':'AcceptedScore'})\
                       .sort_values(by='Difference', ascending = False).reset_index(drop=True)
result_4_2

,Id,Title,MaxScore,AcceptedScore,Difference
0,99713,"In Germany, what are you supposed to do if you...",126,33,93
1,81376,What is way to eat rice with hands in front of...,120,30,90
2,76737,Why do many countries in the world still requi...,118,31,87
3,38177,How do you know if Americans genuinely/literal...,245,166,79
4,90636,Caught speeding 111 Mph (179 km/h) in Californ...,101,25,76
5,81492,Why is international first class much more exp...,90,21,69
6,94320,Strange looking region of France on Google Maps,71,7,64
7,80050,What's an easy way of making my luggage unique...,72,16,56


**4.3.** Porównanie wyników

In [17]:
compare_results(result_4_1,result_4_2)

Ramki danych są tożsame!


<h1></h1>
<h2><center>Zadanie 5</center></h2>
   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Każda odpowiedź w serwisie Stack Exchange może zostać skomentowana przez innych użytkowników. Komentarze są oceniane na podobnej zasadzie co pytania - na podstawie _łapek w górę/łapek w dół_ wyliczany jest _Score_ dla każdego komentarza. 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Zapytanie z zadania piątego zwraca 10 pytań z największą liczbą _Score_ dla komentarzy dodanych przez autora pytania. W ramce danych umieszczone są dwie wartości: _Title_ - treść pytania oraz _CommentsTotalScore_ - suma _Score_ dla komentarzy autora, przy czym sumowane są wartości z komentarzy dodanych tylko pod postem z pytaniem. _Score_ z komentarzy pod odpowiedziami nie jest brany pod uwagę.

```SQL
SELECT
    Posts.Title,
    CmtTotScr.CommentsTotalScore
FROM (
        SELECT
            PostID,
            UserID,
            SUM(Score) AS CommentsTotalScore
        FROM Comments
        GROUP BY PostID, UserID
) AS CmtTotScr
JOIN Posts ON Posts.ID=CmtTotScr.PostID AND Posts.OwnerUserId=CmtTotScr.UserID
WHERE Posts.PostTypeId=1
ORDER BY CmtTotScr.CommentsTotalScore DESC
LIMIT 10
```

**5.1.** pd.read_sql_query

In [18]:
result_5_1 = pd.read_sql_query("""
SELECT
    Posts.Title,
    CmtTotScr.CommentsTotalScore
FROM (
        SELECT
            PostID,
            UserID,
            SUM(Score) AS CommentsTotalScore
        FROM Comments
        GROUP BY PostID, UserID
) AS CmtTotScr
JOIN Posts ON Posts.ID=CmtTotScr.PostID AND Posts.OwnerUserId=CmtTotScr.UserID
WHERE Posts.PostTypeId=1
ORDER BY CmtTotScr.CommentsTotalScore DESC
LIMIT 10
""", conn)
result_5_1

,Title,CommentsTotalScore
0,How to intentionally get denied entry to the U...,75
1,How can I deal with people asking to switch se...,32
2,What is France's traditional costume?,26
3,What's the longest scheduled public bus ride i...,25
4,Can I have a watermelon in hand luggage?,25
5,How does President Trump's travel ban affect n...,25
6,Caught speeding 111 Mph (179 km/h) in Californ...,24
7,Returning US Citizen lost passport in Canada,23
8,Legalities and safety concerns of visiting pro...,20
9,India just demonetized all Rs 500 & 1000 notes...,20


**5.2.** pandas

In [19]:
CmtTotScr = Comments.groupby(['PostId','UserId'])['Score'].agg('sum').reset_index()\
                    .rename(columns={'Score': 'CommentsTotalScore'})
result_5_2 = CmtTotScr.merge(Posts,left_on=['PostId','UserId'],right_on=['Id','OwnerUserId'])\
                      .query('PostTypeId==1').sort_values(by=['CommentsTotalScore','PostId'], ascending=[False,True])\
                      .loc[:,['Title','CommentsTotalScore']].reset_index(drop=True).head(10)
result_5_2

,Title,CommentsTotalScore
0,How to intentionally get denied entry to the U...,75
1,How can I deal with people asking to switch se...,32
2,What is France's traditional costume?,26
3,What's the longest scheduled public bus ride i...,25
4,Can I have a watermelon in hand luggage?,25
5,How does President Trump's travel ban affect n...,25
6,Caught speeding 111 Mph (179 km/h) in Californ...,24
7,Returning US Citizen lost passport in Canada,23
8,Legalities and safety concerns of visiting pro...,20
9,India just demonetized all Rs 500 & 1000 notes...,20


**5.3.** Porównanie wyników

In [20]:
compare_results(result_5_1,result_5_2)

Ramki danych są tożsame!


<h1></h1>
<h2><center>Zadanie 6</center></h2>
   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Użytkownicy Stack Exchange za swoją aktywność mogą dostawać odznaki (_Badges_). Odznaki podzielone są na trzy klasy ze względu na trudność ich zdobycia - brązowe (3), srebrne (2) i złote (1). Zapytanie z zadania szóstego zwraca wszystkich użytkowników (ich Id, nazwę (_DisplayName_), punkty reputacji (_Reputation_), wiek (_Age_) i miejsce zamieszkania (_Location_)), którzy mogą się pochwalić zdobyciem _wartościowych odznak_.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;_Wartościowa odznaka_ to taka, która należy do klasy 1, tj. złota, oraz którą posiada nie mniej niż 2 użytkowników, ale nie więcej niż 10 w całej analizowanej bazie. 

```SQL
SELECT DISTINCT
    Users.Id,
    Users.DisplayName,
    Users.Reputation,
    Users.Age,
    Users.Location
FROM (
        SELECT
            Name, UserID
        FROM Badges
        WHERE Name IN (
                SELECT
                    Name
                FROM Badges
                WHERE Class=1
                GROUP BY Name
                HAVING COUNT(*) BETWEEN 2 AND 10
        )
        AND Class=1
    ) AS ValuableBadges
JOIN Users ON ValuableBadges.UserId=Users.Id
```

**6.1.** pd.read_sql_query()

In [21]:
result_6_1 = pd.read_sql_query("""
SELECT DISTINCT
    Users.Id,
    Users.DisplayName,
    Users.Reputation,
    Users.Age,
    Users.Location
FROM (
        SELECT
            Name, UserID
        FROM Badges
        WHERE Name IN (
                SELECT
                    Name
                FROM Badges
                WHERE Class=1
                GROUP BY Name
                HAVING COUNT(*) BETWEEN 2 AND 10
        )
        AND Class=1
    ) AS ValuableBadges
JOIN Users ON ValuableBadges.UserId=Users.Id
""", conn)
result_6_1

,Id,DisplayName,Reputation,Age,Location
0,108,Ankur Banerjee,31273.0,27.0,"London, UK"
1,19,VMAtm,18556.0,33.0,"Tampa, FL, United States"
2,101,Mark Mayo,121667.0,37.0,"Sydney, New South Wales, Australia"
3,466,iHaveacomputer,8360.0,NaN,Down underer
4,793,mindcorrosive,10531.0,32.0,Bulgaria
5,693,RoflcoptrException,33300.0,NaN,None
6,6669,Relaxed,69405.0,NaN,None
7,1737,Gayot Fow,70237.0,NaN,"London, United Kingdom"
8,39065,Pont,1004.0,NaN,Austria
9,19400,phoog,34342.0,50.0,"New York, NY"


**7.2.** pandas

In [22]:
ValuableBadges = Badges[Badges.loc[:,'Class'] == 1 &
                        Badges.loc[:,'Name'].isin(Badges.query('Class==1').groupby(['Name'])['Id']\
                              .agg('count').reset_index().query('2 <= Id <= 10').loc[:,'Name'])]
result_6_2 = ValuableBadges.merge(Users, left_on = 'UserId', right_on = 'Id')\
                           .loc[:,['Id_y','DisplayName','Reputation','Age','Location']]\
                           .rename(columns={'Id_y': 'Id'}).drop_duplicates().reset_index(drop=True)
result_6_2

,Id,DisplayName,Reputation,Age,Location
0,108,Ankur Banerjee,31273.0,27.0,"London, UK"
1,19,VMAtm,18556.0,33.0,"Tampa, FL, United States"
2,101,Mark Mayo,121667.0,37.0,"Sydney, New South Wales, Australia"
3,466,iHaveacomputer,8360.0,NaN,Down underer
4,793,mindcorrosive,10531.0,32.0,Bulgaria
5,693,RoflcoptrException,33300.0,NaN,NaN
6,6669,Relaxed,69405.0,NaN,NaN
7,1737,Gayot Fow,70237.0,NaN,"London, United Kingdom"
8,39065,Pont,1004.0,NaN,Austria
9,19400,phoog,34342.0,50.0,"New York, NY"


**6.3.** Porównanie wyników

In [23]:
compare_results(result_6_1,result_6_2)

Ramki danych są tożsame!


<h1></h1>
<h2><center>Zadanie 7</center></h2>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Zapytanie z zadania siódmego zwraca 10 pytań (_Title_) z największą liczbą _starych_ głosów typu _łapka w górę_ (_OldVotes_). 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Głos zostaje zaklasyfikowany jako _stary_, jeśli pochodzi z roku 2015 lub wcześniej, natomiast głosy z lat 2016 i 2017 klasyfikowane są jako głosy _nowe_. Dla każdego posta zapytanie najpierw pobiera liczbę głosów _starych_ i _nowych_, a następnie wybiera te posty, które nie mają żadnych głosów sklasyfikowanych jako _nowe_. Zapytanie filtruje posty, aby zwrócić tylko pytania, a następnie wybiera 10 najlepszych pod względem liczby głosów zaklasyfikowanych jako _stare_. Wynik posortowany jest malejąco względem kolumny _OldVotes_.

```SQL
SELECT
    Posts.Title,
    VotesByAge2.OldVotes
FROM Posts
JOIN (
    SELECT
        PostId,
        MAX(CASE WHEN VoteDate = 'new' THEN Total ELSE 0 END) NewVotes,
        MAX(CASE WHEN VoteDate = 'old' THEN Total ELSE 0 END) OldVotes,
        SUM(Total) AS Votes
    FROM (
        SELECT
            PostId,
            CASE STRFTIME('%Y', CreationDate)
                WHEN '2017' THEN 'new'
                WHEN '2016' THEN 'new'
                ELSE 'old'
                END VoteDate,
            COUNT(*) AS Total
        FROM Votes
        WHERE VoteTypeId=2
        GROUP BY PostId, VoteDate
        ) AS VotesByAge
        GROUP BY VotesByAge.PostId
        HAVING NewVotes=0
) AS VotesByAge2 ON VotesByAge2.PostId=Posts.ID
WHERE Posts.PostTypeId=1
ORDER BY VotesByAge2.OldVotes DESC
LIMIT 10
```

**7.1.** pd.read_sql_query

In [24]:
result_7_1 = pd.read_sql_query("""
SELECT
    Posts.Title,
    VotesByAge2.OldVotes
FROM Posts
JOIN (
    SELECT
        PostId,
        MAX(CASE WHEN VoteDate = 'new' THEN Total ELSE 0 END) NewVotes,
        MAX(CASE WHEN VoteDate = 'old' THEN Total ELSE 0 END) OldVotes,
        SUM(Total) AS Votes
    FROM (
        SELECT
            PostId,
            CASE STRFTIME('%Y', CreationDate)
                WHEN '2017' THEN 'new'
                WHEN '2016' THEN 'new'
                ELSE 'old'
                END VoteDate,
            COUNT(*) AS Total
        FROM Votes
        WHERE VoteTypeId=2
        GROUP BY PostId, VoteDate
        ) AS VotesByAge
        GROUP BY VotesByAge.PostId
        HAVING NewVotes=0
) AS VotesByAge2 ON VotesByAge2.PostId=Posts.ID
WHERE Posts.PostTypeId=1
ORDER BY VotesByAge2.OldVotes DESC
LIMIT 10
""", conn)
result_7_1

,Title,OldVotes
0,Which European cities have bike rental station...,39
1,Why do hostels require you to 'rent' bedding?,28
2,What to do with your valuables on a low-cost h...,25
3,Can't check-in to a hotel because I am 18,25
4,What are some good ways to find things to expl...,24
5,Alarm Clock without Noise? To wake up in commo...,24
6,What times of the year are best for visiting F...,23
7,What is the business model of commercial free ...,23
8,Getting work on a cruise ship in order to travel,23
9,Carrying medicines internationally for a friend,23


**7.2.** pandas

In [25]:
Votes.loc[:,'VoteDate'] = pd.to_datetime(Votes.loc[:,"CreationDate"]).apply(lambda x: 'new' if x.year == 2017 or x.year == 2016 else 'old')
VotesByAge = Votes[Votes.loc[:,'VoteTypeId'] == 2].groupby(['PostId','VoteDate'])['Id'].agg('count').reset_index().rename(columns={'Id': 'Total'})
VotesByAge.loc[:,'NewVotes'] = 0
VotesByAge.loc[:,'OldVotes'] = 0
VotesByAge.loc[VotesByAge.loc[:,'VoteDate'] == 'new','NewVotes'] = VotesByAge.loc[:,'Total']
VotesByAge.loc[VotesByAge.loc[:,'VoteDate'] == 'old','OldVotes'] = VotesByAge.loc[:,'Total']
VotesByAge2 = VotesByAge.groupby(['PostId']).agg({'NewVotes': 'max', 'OldVotes': 'max', 'Total': 'sum'}).reset_index()
result_7_2 = Posts[Posts.loc[:,'PostTypeId']==1].merge(VotesByAge2[VotesByAge2.loc[:,'NewVotes']==0], left_on = 'Id', right_on = 'PostId' ).sort_values(by=['OldVotes','PostId'],ascending=[False,True]).loc[:,['Title','OldVotes']].reset_index(drop=True).head(10)
result_7_2

,Title,OldVotes
0,Which European cities have bike rental station...,39
1,Why do hostels require you to 'rent' bedding?,28
2,What to do with your valuables on a low-cost h...,25
3,Can't check-in to a hotel because I am 18,25
4,What are some good ways to find things to expl...,24
5,Alarm Clock without Noise? To wake up in commo...,24
6,What times of the year are best for visiting F...,23
7,What is the business model of commercial free ...,23
8,Getting work on a cruise ship in order to travel,23
9,Carrying medicines internationally for a friend,23


**7.3.** Porównanie wyników

In [26]:
compare_results(result_7_1,result_7_2)

Ramki danych są tożsame!
